In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt    
from collections import defaultdict


def average_of_first_percent(lst, p):
    n = int(len(lst) * p)
    return np.mean(lst[:n])

def average_of_last_percent(lst, p):
    n = int(len(lst) * p)
    return np.mean(lst[-n:])

def find_data_dir_with_mobility(base_dir: str, mobility: float):
    mobility_str = str(mobility).replace(".", "_") + ".txt"
    for dir in os.listdir(base_dir):
        if mobility_str in dir:
            print(f"Found {mobility} in {dir}")
            return os.path.join(base_dir, dir, "machine0")
    
    print(f"Could not find mobility {mobility} in {base_dir}")
    raise ValueError

def inf_dict():
    return defaultdict(inf_dict)

plt.rcParams.update({'font.size': 18})

num_nodes = 48

experiments_dir = ".saved_experiments"

# IID
# experiment_dir = "2024-06-12-plain"

# experiment_dir = "2024-06-20-sol-0.1"
# experiment_dir = "2024-06-20-sol-0.2"
# experiment_dir = "2024-06-20-sol-0.3"
# experiment_dir = "2024-06-20-sol-0.4"
# experiment_dir = "2024-06-15-sol-1.0"

# NIID 
# experiment_dir = "2024-06-17-plain-niid"

# experiment_dir = "2024-06-21-sol-niid-0.1"
# experiment_dir = "2024-06-21-sol-niid-0.2"
# experiment_dir = "2024-06-21-sol-niid-0.3"
# experiment_dir = "2024-06-21-sol-niid-0.4"
experiment_dir = "2024-06-19-sol-niid-1.0"

base_dir = os.path.join(experiments_dir, experiment_dir)

mobility_aware = "sol" in experiment_dir
niid = "niid" in experiment_dir

alpha = None
if mobility_aware:
    alpha = 1.0
    maybe_alpha = float(experiment_dir.split("-")[-1])
    if maybe_alpha <= 1.0 and maybe_alpha >= 0.0: 
        alpha = maybe_alpha
    else:
        raise ValueError(f"Could not find alpha in {experiment_dir}")

num_experiments = 3
experiment_groups ={i : os.path.join(base_dir, str(i)) for i in range(1, num_experiments + 1)}
"""
- All

Performance Effects:
- 0, 20, 40, 60
- 60 80 100
- 0 5

"""

experiments = {
    0.0: "0% High Mobility",
    0.05: "5% High Mobility",
    0.2: "20% High Mobility",
    0.4: "40% High Mobility",
    0.6: "60% High Mobility",
    0.8: "80% High Mobility",
    1.0: "100% High Mobility",
}
statistics = ["test_acc"]
num_rounds = 4000

def load_experiment_data(experiment_data, base_dir):
    for experiment, _ in experiments.items():
        experiment_data[experiment] = {}
        dir = find_data_dir_with_mobility(base_dir, experiment)
        for i in range(num_nodes):
            file_path = os.path.join(dir, f"{i}_results.json")
            if os.path.exists(file_path):
                with open(file_path, "r") as file:
                    content = json.load(file)

                    for statistic, rounds in content.items():
                        if statistic not in experiment_data[experiment]:
                            experiment_data[experiment][statistic] = {}

                        for round, value in rounds.items():
                            round = int(round)
                            if round - 1> num_rounds:
                                continue

                            if round not in experiment_data[experiment][statistic]:
                                experiment_data[experiment][statistic][round] = []
                            experiment_data[experiment][statistic][round].append(value)

def compute_experiment_averages(experiment_data, statistics):
    averages = inf_dict()
    high_mobility_averages = inf_dict() 
    low_mobility_averages = inf_dict()
    for experiment in experiment_data:
        averages[experiment] = inf_dict()
        for statistic in statistics:
            for round, values in experiment_data[experiment][statistic].items():
                averages[experiment][statistic][round] = np.mean(values)

                if round == "10":
                    print(values)
                    print(len(values))

                high_mobility_proportion = experiment
                low_mobility_proportion = 1 - experiment
                high_mobility_averages[experiment][statistic][round] = average_of_first_percent(values, high_mobility_proportion)
                low_mobility_averages[experiment][statistic][round] = average_of_last_percent(values, low_mobility_proportion)
    return averages, high_mobility_averages, low_mobility_averages

def plot_averages(experiment_names, averages, experiment_group="", save_fig=False, new_fig=True, show=True, title="Test Accuracy per Round", ylabel="Average Test Accuracy"):
    if new_fig:
        plt.figure(figsize=(10, 6))

    for experiment, _ in averages.items():
        for statistic in statistics:
            rounds, acc_values = zip(*averages[experiment][statistic].items())
            print(rounds, acc_values)
            plt.plot(rounds, acc_values, label=f"{experiment_names[experiment]}")

    plt.xlabel("Round")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    if save_fig and len(experiment_names) > 1 and niid:
        fig_dir = f"saved_figures/{experiment_dir}"
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)
        

        plt.savefig(f"{fig_dir}/test_accuracy_group_{[str(s) for s in experiments]}_{experiment_group}_{experiment_dir}.pdf")
    if show:
        plt.show()


grouped_experiments_data = inf_dict()
for experiment_group, dir in experiment_groups.items():
    experiment_data = grouped_experiments_data[experiment_group]
    load_experiment_data(experiment_data, dir)

grouped_averages = inf_dict()
grouped_high_mobility_averages = inf_dict()
grouped_low_mobility_averages = inf_dict()
for experiment_group, experiment_data in grouped_experiments_data.items():
    averages, high_mobility_averages, low_mobility_averages = compute_experiment_averages(experiment_data, statistics)
    grouped_averages[experiment_group] = averages
    grouped_high_mobility_averages[experiment_group] = high_mobility_averages
    grouped_low_mobility_averages[experiment_group] = low_mobility_averages


for experiment_group, averages in grouped_averages.items():
    plot_averages(experiments, averages)

for experiment_group, high_mobility_averages in grouped_high_mobility_averages.items():
    plot_averages(experiments, high_mobility_averages)

for experiment_group, low_mobility_averages in grouped_low_mobility_averages.items():
    plot_averages(experiments, low_mobility_averages)


In [ ]:
total_averages = inf_dict()
total_high_mobility_averages = inf_dict()
total_low_mobility_averages = inf_dict()

for experiment_group, averages in grouped_averages.items():
    for experiment, _ in averages.items():
        for statistic in statistics:
            print(experiment, statistic, averages[experiment][statistic].items())
            print(experiment, statistic, len(averages[experiment][statistic].items()))
            for round, value in averages[experiment][statistic].items():
                if round not in total_averages[experiment][statistic]:
                    total_averages[experiment][statistic][round] = []
                total_averages[experiment][statistic][round].append(value)

for experiment_group, high_mobility_averages in grouped_high_mobility_averages.items():
    for experiment, _ in high_mobility_averages.items():
        for statistic in statistics:
            for round, value in high_mobility_averages[experiment][statistic].items():
                if round not in total_high_mobility_averages[experiment][statistic]:
                    total_high_mobility_averages[experiment][statistic][round] = []
                total_high_mobility_averages[experiment][statistic][round].append(value)

for experiment_group, low_mobility_averages in grouped_low_mobility_averages.items():
    for experiment, _ in low_mobility_averages.items():
        for statistic in statistics:
            for round, value in low_mobility_averages[experiment][statistic].items():
                if round not in total_low_mobility_averages[experiment][statistic]:
                    total_low_mobility_averages[experiment][statistic][round] = []
                total_low_mobility_averages[experiment][statistic][round].append(value)

for statistic in statistics:
    for experiment, _ in total_averages.items():
        for round, values in total_averages[experiment][statistic].items():
            total_averages[experiment][statistic][round] = np.mean(values)

    for experiment, _ in total_high_mobility_averages.items():
        for round, values in total_high_mobility_averages[experiment][statistic].items():
            total_high_mobility_averages[experiment][statistic][round] = np.mean(values)

    for experiment, _ in total_low_mobility_averages.items():
        for round, values in total_low_mobility_averages[experiment][statistic].items():
            total_low_mobility_averages[experiment][statistic][round] = np.mean(values)

print(total_averages)
plot_averages(experiments, total_averages, "Total", save_fig=True)


In [ ]:
yes_mob_aware_key = f"mobility_aware_niid_{niid}_{alpha}"
non_mob_aware_key = f"non_mobility_aware_niid_{niid}"

mob_aware_key = yes_mob_aware_key if mobility_aware else non_mob_aware_key

def save_test_accuracy_averages(mobility_aware, experiments):
    file_name = "saved_figures/test_accuracy_avg.json"
    for experiment in experiments:
        statistic = "test_acc"
        exp_key = str(experiment)
        with open(file_name) as file:
            content = {}
            try:
                content = json.load(file)
            except Exception as e:
                print("error loading file", e)
            
            if exp_key not in content:
                content[exp_key] = {}

            key = mob_aware_key
            
            if key not in content[exp_key]:
                content[exp_key][key] = {}

            content[exp_key][key] = averages[experiment][statistic]

            if 'final_avg' not in content:
                content['final_avg'] = {}

            if exp_key not in content['final_avg']:
                content['final_avg'][exp_key] = {}

            last_round = max(averages[experiment][statistic].keys())
            content['final_avg'][exp_key][key] = averages[experiment][statistic][last_round]

            with open(file_name, "w") as file:
                json.dump(content, file)


def compare_test_accuracy_averages():
    if alpha is None:
        print("No alpha, not comparing")
        return []

    file_name = "saved_figures/test_accuracy_avg.json"
    diffs = inf_dict()
    statistic = "test_acc"
    with open(file_name) as file:
        content = json.load(file)
        for experiment in experiments:
            exp_key = str(experiment)
            
            if yes_mob_aware_key not in content[exp_key] or non_mob_aware_key not in content[exp_key]:
                raise ValueError("Missing data for comparison, experiment", experiment)

            key = "diff"
            mobility_aware_exp = content[exp_key][yes_mob_aware_key]
            non_mobility_aware_exp = content[exp_key][non_mob_aware_key]

            if len(mobility_aware_exp) != len(non_mobility_aware_exp):
                raise ValueError("Different number of rounds for mobility aware and non mobility aware experiments")
            
            diff_per_round = inf_dict()
            for round in mobility_aware_exp:
                diff_per_round[int(round)] = non_mobility_aware_exp[round] - mobility_aware_exp[round]

            content[exp_key][key] = diff_per_round
            print(experiment, diff_per_round)

            diffs[experiment][statistic] = diff_per_round

        diff_averages = inf_dict()
        for experiment in experiments:
            diff_avg = np.mean(list(diffs[experiment][statistic].values()))
            diff_averages[experiment] = diff_avg

        content[f"diff_averages_niid_{niid}_alpha_{alpha}"] = diff_averages
        
        with open(file_name, "w") as file:
            json.dump(content, file)

    return diffs


def compare_improvement_test_accuracy():
    file_name = "saved_figures/test_accuracy_avg.json"
    diffs = inf_dict()
    statistic = "test_acc"
    with open(file_name) as file:
        content = json.load(file)
        
        experiments_ordered = sorted(experiments.keys())
        baseline = experiments_ordered[0]

        baseline_exp = content[str(baseline)][mob_aware_key]
        for experiment in experiments_ordered:
            exp_key = str(experiment)

            key = "compare_baseline"
            exp = content[exp_key][mob_aware_key]

            if len(exp) != len(baseline_exp):
                raise ValueError("Different number of rounds for baseline and experiment")

            diff_per_round = inf_dict()
            for round in baseline_exp:
                diff_per_round[int(round)] = exp[round] - baseline_exp[round]

            if key not in content[exp_key]:
                content[exp_key][key] = {}

            content[exp_key][key][mob_aware_key] = diff_per_round
            print(experiment, diff_per_round)

            diffs[experiment][statistic] = diff_per_round

        diff_averages = inf_dict()
        final_diff_average = inf_dict()
        for experiment in experiments:
            diff_avg = np.mean(list(diffs[experiment][statistic].values()))
            diff_averages[experiment] = diff_avg

            final_round = max(diffs[experiment][statistic].keys())
            final_diff_average[experiment] = diffs[experiment][statistic][final_round]

        compare_baseline_key = "compare_baseline_avg"
        if compare_baseline_key not in content:
            content[compare_baseline_key] = {}
        content[compare_baseline_key][mob_aware_key] = diff_averages

        compare_final_key = "compare_baseline_final_avg"
        if compare_final_key not in content:
            content[compare_final_key] = {}
        content[compare_final_key][mob_aware_key] = final_diff_average
        
        with open(file_name, "w") as file:
            json.dump(content, file)

    return diffs





save_test_accuracy_averages(mobility_aware, experiments)
if alpha is not None:
    diff_averages = compare_test_accuracy_averages()
    plot_averages(experiments, diff_averages, "Diff", title="Test Accuracy Difference per Round\nPlain Average - Mob Aware", ylabel="Average Test Accuracy Difference")

compare_baseline_diffs = compare_improvement_test_accuracy()
plot_averages(experiments, compare_baseline_diffs, "Compare Baseline", title="Test Accuracy Difference per Round comp. Baseline", ylabel="Average Test Accuracy Difference")

In [ ]:
def plot_high_low_averages(high_average, low_average, statistic, experiment, prop_solution):
    plt.figure(figsize=(10, 6))

    rounds, acc_values = zip(*high_average[experiment][statistic].items())
    plt.plot(rounds, acc_values, label=f"High Mobility")

    rounds, acc_values = zip(*low_average[experiment][statistic].items())
    plt.plot(rounds, acc_values, label=f"Low Mobility")

    plt.xlabel("Round")
    plt.ylabel("Average Test Accuracy")
    plt.title(f"Test Accuracy per Round, {experiment * 100}% High-Mobility Clients,\n {'Not' if not prop_solution else ''} Mobility-Aware α={alpha}")
    plt.legend()
    plt.grid(True)
    if niid:
        plt.savefig(f"saved_figures/test_accuracy_high_low_{experiment}_mobility_aware_{prop_solution}_niid_{niid}_{alpha}.pdf")
    plt.show()

def get_high_low_diff(high_average, low_average, statistic):
    differences = {}
    for round in high_average[statistic]:
        differences[round] = high_average[statistic][round] - low_average[statistic][round]
    return differences, np.array(list(differences.values()))

def save_diff_analysis(diff_per_round, mean, std, max, min, experiment, high_final, low_final):
    file_name = "saved_figures/diff_analysis_mobility_aware.json"
    experiment = str(experiment)
    with open(file_name) as file:
        content = {}
        try:
            content = json.load(file)
        except Exception as e:
            print("error loading file", e)
        
        if experiment not in content:
            content[experiment] = {}
        
        if mob_aware_key not in content[experiment]:
            content[experiment][mob_aware_key] = {}

        content[experiment][mob_aware_key] = {
            "mean": mean,
            "std": std,
            "max": max,
            "min": min,
            "low_final": low_final,
            "high_final": high_final,
            "diff_per_round": diff_per_round,
        }

        with open(file_name, "w") as file:
            json.dump(content, file)

for experiment in experiments:
    high_mobility_averages = total_high_mobility_averages
    low_mobility_averages = total_low_mobility_averages

    plot_high_low_averages(high_mobility_averages, low_mobility_averages, "test_acc", experiment, mobility_aware)
    diff_dict, diff = get_high_low_diff(high_mobility_averages[experiment], low_mobility_averages[experiment], "test_acc")
    print(diff_dict)
    diff.mean(), diff.std(), diff.max(), diff.min()

    high_final = list(high_mobility_averages[experiment]["test_acc"].values())[-1]
    low_final = list(low_mobility_averages[experiment]["test_acc"].values())[-1]
    save_diff_analysis(diff_dict, diff.mean(), diff.std(), diff.max(), diff.min(), experiment, high_final, low_final)